In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import threading
import time
import matplotlib.pyplot as plt
import numpy as np
import threading
import time
import socket
import json
import torch
import torch.nn as nn
import random
import torch.nn.functional as F


In [2]:
N=5
GAMMA=0.99
MAX_STEPS=100
EPS=0.99
ALPHA=0.01
FREQ_UPDATE=30
replay_memory1=[]
replay_memory2=[]
dt=0.1
BATCH_SIZE=16
critical_angle=75

In [3]:
device="cuda"
class Agent(nn.Module):
    def __init__(self,input_size):
        super(Agent, self).__init__()
        self.bn_input=nn.BatchNorm1d(input_size)

        self.fc1=nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2=nn.Linear(64, 64)
        self.bn2 = nn.BatchNorm1d(64)

        self.output_fc=nn.Linear(64, 3)
    def forward(self, x):
        out=self.bn_input(x)
        out=F.relu(self.fc1(out))
        out=self.bn1(out)
        out=F.relu(self.fc2(out))
        out=self.bn2(out)
        out=self.output_fc(out)
        return out
    def act(self,obs):
        self.eval()
        q_values=self(obs.to(device)).to('cpu')
        max_q_index=torch.argmax(q_values)
        action=max_q_index.detach().item()
        self.train()
        return action
    

policy_model1 = Agent(input_size=4).to(device)
target_model1 = Agent(input_size=4).to(device)
target_model1.load_state_dict(policy_model1.state_dict())

policy_model2 = Agent(input_size=4).to(device)
target_model2 = Agent(input_size=4).to(device)
target_model2.load_state_dict(policy_model2.state_dict())


optimizer1=torch.optim.Adam(policy_model1.parameters(), lr=5e-3,weight_decay=5e-5)
optimizer2=torch.optim.Adam(policy_model2.parameters(), lr=5e-3,weight_decay=5e-5)

In [4]:

class PIDController:
    def __init__(self, kp, ki, kd, setpoint):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.setpoint = setpoint
        self.integral = 0
        self.previous_error = 0

    def update(self, current_value, dt):
        error = self.setpoint - current_value
        self.integral += error * dt
        derivative = (error - self.previous_error) / dt
        output = (self.kp * error) + (self.ki * self.integral) + (self.kd * derivative)
        self.previous_error = error
        return output

class VirtualMotorPropeller:
    def __init__(self, R, Ke, Kt, thrust_coefficient, voltage_limit):
        self.angular_velocity = 0.0
        self.moment_inertia=0.012
        self.R = R
        self.Ke = Ke
        self.Kt = Kt
        self.thrust_coefficient = thrust_coefficient
        self.voltage_limit = voltage_limit
        self.current = 0.0

    def apply_voltage(self, voltage, dt):
        voltage = max(min(voltage, self.voltage_limit), -self.voltage_limit)
        back_emf = self.Ke * self.angular_velocity
        self.current = (voltage - back_emf) / self.R
        torque = self.Kt * self.current
        angular_acceleration = torque/self.moment_inertia
        self.angular_velocity += angular_acceleration * dt


    def get_thrust(self):
        return self.thrust_coefficient * (self.angular_velocity ** 2)
# Seesaw Model Class
class Seesaw:
    def __init__(self, J, arm_length, damping):
        self.angle = 0.0
        self.angular_velocity = 0.0
        self.J = J 
        self.arm_length = arm_length
        self.damping = damping 

    def apply_forces(self, thrust_left, thrust_right, dt):
        net_torque = (self.arm_length/2) * (thrust_right - thrust_left)
        gravity_torque = 0.3 * 9.8 * (self.arm_length / 2) * np.sin(self.angle)
        total_torque = net_torque - gravity_torque
        angular_acceleration = (total_torque - self.damping * self.angular_velocity) / self.J
        self.angular_velocity += angular_acceleration * dt
        self.angle += self.angular_velocity * dt

    def get_angle_deg(self):
        return self.angle * (180.0 / np.pi)

class Gyroscope:
    def __init__(self, noise_std=0.0):
        self.noise_std = noise_std

    def read_angle(self, true_angle):
        noise = np.random.normal(0, self.noise_std)
        return true_angle + noise

In [5]:
def training_model1(replay_memory):
    replay_memory=torch.tensor(replay_memory, dtype=torch.float32)
    state=replay_memory[:,:4]
    next_state=replay_memory[:,4:8]
    rewards=replay_memory[:,8].unsqueeze(1)
    actions=replay_memory[:,9].to(torch.int64).unsqueeze(1)
    dones=replay_memory[:,10].unsqueeze(1)
    target_q_values=target_model1(next_state.to(device)).to('cpu')
    max_target_q_values=target_q_values.max(dim=1,keepdim=True)[0]
    targets=rewards+GAMMA*(1-dones)*max_target_q_values

    q_values=policy_model1(state.to(device)).to('cpu')
    action_q_values=torch.gather(input=q_values,dim=1,index=actions)
    loss=nn.functional.mse_loss(action_q_values, targets)
    optimizer1.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(policy_model1.parameters(), max_norm=1.0)
    optimizer1.step()
    torch.cuda.empty_cache()


def training_model2(replay_memory):
    replay_memory=torch.tensor(replay_memory, dtype=torch.float32)
    state=replay_memory[:,:4]
    next_state=replay_memory[:,4:8]
    rewards=replay_memory[:,8].unsqueeze(1)
    actions=replay_memory[:,9].to(torch.int64).unsqueeze(1)
    dones=replay_memory[:,10].unsqueeze(1)
    target_q_values=target_model2(next_state.to(device)).to('cpu')
    max_target_q_values=target_q_values.max(dim=1,keepdim=True)[0]
    targets=rewards+GAMMA*(1-dones)*max_target_q_values

    q_values=policy_model2(state.to(device)).to('cpu')
    action_q_values=torch.gather(input=q_values,dim=1,index=actions)
    loss=nn.functional.mse_loss(action_q_values, targets)
    optimizer2.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(policy_model2.parameters(), max_norm=1.0)
    optimizer2.step()
    torch.cuda.empty_cache()

In [6]:
def initialize_environment():

    R = 0.5
    Ke = 0.02
    Kt = 0.02
    thrust_coefficient = 1e-5
    voltage_limit = 24.0

    J = 0.012
    arm_length = 0.4
    damping = 0.05

    left_motor = VirtualMotorPropeller(R, Ke, Kt, thrust_coefficient, voltage_limit)
    right_motor = VirtualMotorPropeller(R, Ke, Kt, thrust_coefficient, voltage_limit)
    seesaw = Seesaw(J, arm_length, damping)
    gyro = Gyroscope(noise_std=0.0)
    return left_motor,right_motor,seesaw,gyro,voltage_limit

left_motor,right_motor,seesaw,gyro,voltage_limit=initialize_environment()

In [8]:
voltage_left=0
voltage_right=0
def run_step_simulation(control_signal_right,control_signal_left):
    global voltage_limit
    global dt
    global seesaw
    global left_motor
    global right_motor
    global voltage_left
    global voltage_right
    voltage_right = np.clip(voltage_right+control_signal_right, 0, voltage_limit)
    voltage_left = np.clip(voltage_left+control_signal_left, 0, voltage_limit)

    left_motor.apply_voltage(voltage_left, dt)
    right_motor.apply_voltage(voltage_right, dt)

    thrust_left = left_motor.get_thrust()
    thrust_right = right_motor.get_thrust()

    seesaw.apply_forces(thrust_left, thrust_right, dt)


def step(state):
    global gyro
    target_angle_degree=state[0]
    state=torch.tensor([state], dtype=torch.float32)

    if random.uniform(0,1)>EPS:
        action1=policy_model1.act(state)
        action2=policy_model2.act(state)
    else:
        action1=random.randint(0,2)
        action2=random.randint(0,2)

    if action1==0:
        control_signal_right=-12
    elif action1==1:
        control_signal_right=12
    else:
        control_signal_right=0
    
    if action2==0:
        control_signal_left=-12
    elif action2==1:
        control_signal_left=12
    else:
        control_signal_left=0

    run_step_simulation(control_signal_right=control_signal_right,control_signal_left=control_signal_left)
    state=[target_angle_degree,gyro.read_angle(seesaw.get_angle_deg()),left_motor.get_thrust(),right_motor.get_thrust()]
    reward=-abs(target_angle_degree-gyro.read_angle(seesaw.get_angle_deg()))
    return state,reward,action1,reward,action2
def play_episode(state):
    global gyro
    for i in range(MAX_STEPS):
        old_state=state.copy()
        state,rew_t1,action_t1, rew_t2, action_t2=step(state)

        obs1=old_state.copy()
        obs1.extend(state)
        obs1.append(rew_t1)
        obs1.append(action_t1)

        obs2=old_state.copy()
        obs2.extend(state)
        obs2.append(rew_t2)
        obs2.append(action_t2)

        if i==MAX_STEPS-1 or abs(gyro.read_angle(seesaw.get_angle_deg()))>=critical_angle:
            obs1.append(1)
            obs2.append(1)
            replay_memory1.append(obs1)
            replay_memory2.append(obs2)
        else:
            obs1.append(0)
            obs2.append(0)
            replay_memory1.append(obs1)
            replay_memory2.append(obs2)
    return state,rew_t1,rew_t2


In [ ]:
number_episodes=100000
reward_avg_logs1=[0]
reward_avg_logs2=[0]
for i in range(number_episodes):
    left_motor,right_motor,seesaw,gyro,voltage_limit=initialize_environment()
    target_angle_deg=random.uniform(-60,60)
    measured_angle_deg = gyro.read_angle(seesaw.get_angle_deg())

    state=[target_angle_deg,measured_angle_deg,left_motor.get_thrust(),right_motor.get_thrust()]
    state,rew_t1,rew_t2=play_episode(state)
    EPS=EPS*0.99999
    reward_avg_logs1.append((len(reward_avg_logs1)*reward_avg_logs1[-1]+rew_t1)/(len(reward_avg_logs1)+1))
    reward_avg_logs2.append((len(reward_avg_logs2)*reward_avg_logs2[-1]+rew_t2)/(len(reward_avg_logs2)+1))
    training_model1(replay_memory=random.sample(replay_memory1, BATCH_SIZE))
    training_model2(replay_memory=random.sample(replay_memory2, BATCH_SIZE))

    if (i+1)%FREQ_UPDATE==0:
        target_model1.load_state_dict(policy_model1.state_dict())
        target_model2.load_state_dict(policy_model2.state_dict())
        print("Last reward is:",reward_avg_logs2[-1]," and epsilon= ",EPS)
        print(i)


In [13]:

torch.save(policy_model1.state_dict(), 'agent1.pth')
torch.save(policy_model2.state_dict(), 'agent2.pth')

